# Budovanie s modelmi Mistral

## Úvod

Táto lekcia pokryje:
- Preskúmanie rôznych modelov Mistral
- Pochopenie prípadov použitia a scenárov pre každý model
- Ukážky kódu zobrazujúce jedinečné vlastnosti každého modelu.


## Modely Mistral

V tejto lekcii preskúmame 3 rôzne modely Mistral:  
**Mistral Large**, **Mistral Small** a **Mistral Nemo**.

Každý z týchto modelov je dostupný zadarmo na trhu modelov Github. Kód v tomto notebooku bude používať tieto modely na spustenie kódu. Tu sú ďalšie podrobnosti o používaní modelov Github na [prototypovanie s AI modelmi](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 je v súčasnosti vlajkový model od Mistral a je navrhnutý pre podnikové použitie.

Model je vylepšením pôvodného Mistral Large a ponúka
- Väčšie kontextové okno - 128k vs 32k
- Lepší výkon v matematických a programovacích úlohách - priemerná presnosť 76,9 % vs 60,4 %
- Zvýšený viacjazyčný výkon - jazyky zahŕňajú: angličtinu, francúzštinu, nemčinu, španielčinu, taliančinu, portugalčinu, holandčinu, ruštinu, čínštinu, japončinu, kórejčinu, arabčinu a hindčinu.

Vďaka týmto vlastnostiam Mistral Large vyniká v
- *Retrieval Augmented Generation (RAG)* - vďaka väčšiemu kontextovému oknu
- *Volanie funkcií* - tento model má natívne volanie funkcií, ktoré umožňuje integráciu s externými nástrojmi a API. Tieto volania môžu byť vykonávané paralelne alebo jedno po druhom v sekvenčnom poradí.
- *Generovanie kódu* - tento model vyniká v generovaní Python, Java, TypeScript a C++.


### Príklad RAG pomocou Mistral Large 2


V tomto príklade používame Mistral Large 2 na spustenie vzoru RAG nad textovým dokumentom. Otázka je napísaná v kórejčine a pýta sa na aktivity autora pred vysokou školou.

Používa model Cohere Embeddings na vytvorenie vektorových reprezentácií textového dokumentu aj otázky. Pre tento príklad používa balík faiss v Pythone ako vektorové úložisko.

Výzva odoslaná modelu Mistral obsahuje otázky aj získané časti textu, ktoré sú podobné otázke. Model potom poskytne odpoveď v prirodzenom jazyku.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small je ďalší model v rodine modelov Mistral v kategórii premier/enterprise. Ako názov napovedá, tento model je Malý jazykový model (SLM). Výhody použitia Mistral Small sú: 
- Úspora nákladov v porovnaní s Mistral LLM, ako sú Mistral Large a NeMo - zníženie ceny o 80 %
- Nízka latencia - rýchlejšia odozva v porovnaní s Mistral LLM
- Flexibilný - môže byť nasadený v rôznych prostrediach s menšími obmedzeniami na požadované zdroje. 


Mistral Small je skvelý pre: 
- Úlohy založené na texte, ako je zhrnutie, analýza sentimentu a preklad. 
- Aplikácie, kde sa často vykonávajú požiadavky vďaka jeho nákladovej efektívnosti 
- Úlohy s nízkou latenciou v kóde, ako je kontrola a návrhy kódu 


## Porovnanie Mistral Small a Mistral Large

Ak chcete zobraziť rozdiely v latencii medzi Mistral Small a Large, spustite nižšie uvedené bunky.

Mali by ste vidieť rozdiel v časoch odozvy medzi 3-5 sekundami. Tiež si všimnite dĺžky a štýl odpovedí na rovnaký prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

V porovnaní s ďalšími dvoma modelmi diskutovanými v tejto lekcii je Mistral NeMo jediný bezplatný model s licenciou Apache2.

Je považovaný za vylepšenie predchádzajúceho open source LLM od Mistral, Mistral 7B.

Niektoré ďalšie vlastnosti modelu NeMo sú:

- *Efektívnejšia tokenizácia:* Tento model používa tokenizér Tekken namiesto bežnejšieho tiktoken. To umožňuje lepší výkon v rôznych jazykoch a kóde.

- *Doladenie:* Základný model je dostupný na doladenie. To poskytuje väčšiu flexibilitu pre prípady použitia, kde môže byť doladenie potrebné.

- *Nativne volanie funkcií* - Rovnako ako Mistral Large, tento model bol trénovaný na volanie funkcií. To ho robí jedinečným ako jeden z prvých open source modelov, ktoré to umožňujú.


## Mistral NeMo

V porovnaní s ďalšími dvoma modelmi diskutovanými v tejto lekcii je Mistral NeMo jediný bezplatný model s licenciou Apache2.

Je považovaný za vylepšenie predchádzajúceho open source LLM od Mistral, Mistral 7B.

Niektoré ďalšie vlastnosti modelu NeMo sú:

- *Efektívnejšia tokenizácia:* Tento model používa tokenizér Tekken namiesto bežnejšie používaného tiktoken. To umožňuje lepší výkon v rôznych jazykoch a kóde.

- *Doladenie:* Základný model je dostupný na doladenie. To poskytuje väčšiu flexibilitu pre prípady použitia, kde môže byť doladenie potrebné.

- *Nativne volanie funkcií* - Rovnako ako Mistral Large, tento model bol trénovaný na volanie funkcií. To ho robí jedinečným ako jeden z prvých open source modelov, ktoré to umožňujú.


### Porovnanie tokenizérov

V tomto príklade sa pozrieme na to, ako Mistral NeMo spracováva tokenizáciu v porovnaní s Mistral Large.

Oba príklady používajú rovnaký prompt, ale mali by ste vidieť, že NeMo vracia menej tokenov v porovnaní s Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Učenie tu nekončí, pokračujte na ceste

Po dokončení tejto lekcie si pozrite našu [kolekciu učenia o generatívnej AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), aby ste naďalej zvyšovali svoje znalosti o generatívnej AI!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zrieknutie sa zodpovednosti**:
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, majte prosím na pamäti, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
